In [13]:
!pip install -r requirements.txt

In [14]:
import pandas as pd
import numpy as np

In [15]:
table_production = pd.read_csv("1- producao-de-plastico-global.csv")
table_polution = pd.read_csv("5- poluicao-agua-cidades.csv")

Conexão com banco



In [16]:
import json

import oracledb

with open("credenciais.txt", 'r') as f:
    credenciais = {}
    for line in f:
        key, value = line.strip().split("=")
        credenciais[key] = value


    USER = credenciais["user"]
    PASS = credenciais["pass"]


   
    dsnStr = oracledb.makedsn("oracle.fiap.com.br", 1521, "ORCL")
    
    connection = oracledb.connect(user=USER, password=PASS, dsn=dsnStr)

Função Inicial


In [17]:

def inicio():
    linha = "____________________________________________________________________\n"
    print(linha)
    print("Bem Vindo a ______ o que deseja fazer?")
    print("1 - Deseja adicionar as informações da planilha no Banco?")
    print("2 - Deseja mostrar as informações da última planilha adicionada?")
    print("3 - Sair")
    print(linha)
    verify()


Função para verificação de número

In [18]:
def verify():
    global sair
    try: 
        global escolha
        escolha = int(input("Digite o número que deseja:"))
        print("O número escolhido foi {}. Executando...\n\n".format(escolha))
        
    except ValueError:
        print("Oopss O valor não é um número. Tente de novo")

In [22]:
def inserir():
    connection = oracledb.connect(user=USER, password=PASS, dsn=dsnStr)
    cursor = connection.cursor()
    option = input("Deseja adicionar as tabelas atuais ao Banco de Dados?").upper()
    if option == "SIM":
        table_production = pd.read_csv("1- producao-de-plastico-global.csv")
        table_polution = pd.read_csv("5- poluicao-agua-cidades.csv")
        
        for entidade, ano, PP in table_production:
            sql = "INSERT INTO T_PRODUCTION (entidade, ano, Produção Anual de Plástico) VALUES (:1, :2, :3)"
            cursor.execute(sql, [entidade, ano, PP])
            connection.commit()
    

In [20]:
print(table_production.head())

  Entidade   Ano  Produção Anual de Plástico
0    Mundo  1950                     2000000
1    Mundo  1951                     2000000
2    Mundo  1952                     2000000
3    Mundo  1953                     3000000
4    Mundo  1954                     3000000


In [1]:
import oracledb
import pandas as pd

def iniciar():
    linha = "____________________________________________________________________\n"
    print(linha)
    print("Bem Vindo a ______ o que deseja fazer?")
    print("1 - Deseja adicionar a planilha inteira?")
    print("2 - Deseja adicionar uma informação?")
    print("3 - Deseja atualizar alguma tabela?")
    print("4 - Deseja visualizar alguma tabela?")
    print("5 - Deseja excluir alguma tabela?")
    print("6 - Sair")
    print(linha)
    verificar()

def verificar():
    global escolha
    try: 
        escolha = int(input("Digite o número que deseja:"))
        print("O número escolhido foi {}. Executando...\n\n".format(escolha))
        opcoes()
    except ValueError:
        print("Oopss O valor não é um número. Tente de novo")

def opcoes():
    if escolha == 1:
        planilha()
    elif escolha == 2:
        inserir()
    elif escolha == 3:
        read()
    elif escolha == 4:
        update()
    elif escolha == 5:
        delete()
    elif escolha == 6:
        print("Sistema será fechado!")
        exit(0)  
    else:
        print("Escolha um número válido")
        iniciar()

def planilha():
    with oracledb.connect(user=USER, password=PASS, dsn=dsnStr) as connection:
        cursor = connection.cursor()
        escolha_user = input("Deseja adicionar as tabelas atuais ao Banco de Dados? (SIM/NÃO)").upper()
        if escolha_user == "SIM":
            try:
                table_production = pd.read_csv("1- producao-de-plastico-global.csv")
                table_polution = pd.read_csv("5- poluicao-agua-cidades.csv")
                table_polution.dropna(inplace=True)
                for _, row in table_production.iterrows():
                    Entidade = row['Entidade']
                    Ano = row['Ano']
                    Produção_Anual_Plástico = row["Produção_Anual_de_Plástico"]
                    sql = "INSERT INTO T_PRODUCTION (entidade, ano, Produção_Anual_Plástico) VALUES (:1, :2, :3)"
                    cursor.execute(sql, [Entidade, Ano, Produção_Anual_Plástico])
                    
                for _, row in table_polution.iterrows():
                    Cidade = row['Cidade']
                    Região = row['Região']
                    Entidade = row["Entidade"]
                    Qualidade_do_Ar = row["Qualidade do Ar"]
                    Poluição_da_Água = row["Poluição da Água"]
                    sql = "INSERT INTO T_POLLUTION (Cidade, Região, Entidade, Qualidade_do_Ar, Poluição_da_Água) VALUES (:1, :2, :3, :4, :5)" 
                    cursor.execute(sql,[Cidade,Região,Entidade,Qualidade_do_Ar,Poluição_da_Água])

                connection.commit()
                
                print("Sucesso! As tabelas foram incluídas no Banco de Dados")
            except oracledb.DatabaseError as e:
                error, = e.args
                print(f"Ocorreu um erro: {error.code} - {error.message}")
            finally:
                cursor.close()
                resposta_user = input("Deseja fazer mais alguma ação? (SIM/NÃO)").upper()
                if resposta_user == 'SIM':
                    return inicio()
                else:
                    return final()
                

def inserir():
    connection = oracledb.connect(user=USER, password=PASS, dsn=dsnStr)
    cursor = connection.cursor()

    entrada = input("Qual tabela você deseja adicionar 1 item - Produção de Plástico (PP) ou Poluição da água(PA)").lower()
    
    if entrada == 'pp':
        try:
            Entidade = input("Digite a entidade: ")
            Ano = input("Digite o ano: ")
            Produção_Anual_Plástico = input("Digite a Produção Anual Plástico: ")
            sql = "INSERT INTO T_PRODUCTION (entidade, ano, Produção_Anual_Plástico) VALUES (:1, :2, :3)"
            cursor.execute(sql, [Entidade, Ano, Produção_Anual_Plástico])
            connection.commit()
            print("A nova informação foi concluída!")
        
        except oracledb.DatabaseError as e:
            error, = e.args
            print(f"An error occurred: {error.code} - {error.message}")
        finally:
            cursor.close()
            resposta_user = input("Deseja fazer mais alguma ação? (SIM/NÃO)").upper()
            if resposta_user == 'SIM':
                return inicio()
            else:
                return final()
            
    elif entrada == "pa":
        try:
            Cidade = input("Digite a Cidade: ")
            Região = input("Digite o Região: ")
            Entidade = input("Digite a Entidade: ")
            Qualidade_do_Ar = input("Digite a Qualidade do Ar: ")
            Poluição_da_Água = input("Digite a Poluição da Água: ")
            sql = "INSERT INTO T_POLLUTION (Cidade, Região, Entidade, Qualidade_do_Ar, Poluição_da_Água) VALUES (:1, :2, :3, :4, :5)"
            cursor.execute(sql, [Cidade, Região, Entidade, Qualidade_do_Ar, Poluição_da_Água])
            connection.commit()
            print("A nova informação foi concluída!")
        
        except oracledb.DatabaseError as e:
            error, = e.args
            print(f"An error occurred: {error.code} - {error.message}")
        finally:
            cursor.close()
            resposta_user = input("Deseja fazer mais alguma ação? (SIM/NÃO)").upper()
            if resposta_user == 'SIM':
                return inicio()
            else:
                return final()
    else:
        print("Escolha uma opção válida!")
        return inicio()
            

def delete():
    connection = oracledb.connect(user=USER, password=PASS, dsn=dsnStr)
    cursor = connection.cursor()
    choice = input("Você quer excluir a tabela inteira ou apenas um item? (inteiro/um)").lower()
    
    try:
        if choice == 'inteiro':
            table = input("Qual o nome da tabela?")
            cursor.execute(f"DROP TABLE \"{table}\";")
            connection.commit()
            
        elif choice == 'um':
            escolha_table = input("Qual o nome da tabela que você deseja excluir? (Produção de Plástico ou Poluição da água)").lower()
            
            if escolha_table == 'pp':
                ano_pk = input("Digite o ano que gostaria de excluir")
                cursor.execute("DELETE FROM T_PRODUCTION WHERE ano = :2", [ano_pk])
                connection.commit()
                print(f"O ano de número: {ano_pk} foi excluído!")
                
            elif escolha_table == 'pa':
                cidade_pk = input("Digite a cidade que gostaria de excluir")
                cursor.execute("DELETE FROM T_POLLUTION WHERE cidade = :1", [cidade_pk])
                connection.commit()
                print(f"A informação de {cidade_pk} foi excluído!")
            else:
                print("Escolha uma opção aceitável!")
                return inicio()
        else:
            print("Tente novamente!") 
            return inicio()
    except Exception as e:
        print(f"Erro ao tentar excluir a tabela: {e}")
    finally:
        cursor.close()
        resposta_user = input("Deseja fazer mais alguma ação? (SIM/NÃO)").upper()
        if resposta_user == 'SIM':
            return inicio()
        else:
            return final()
            
def read():
    connection = oracledb.connect(user=USER, password=PASS, dsn=dsnStr)
    cursor = connection.cursor()
    try:
        # Lendo dados da tabela T_POLLUTION
        cursor.execute("SELECT * FROM T_POLLUTION")
        rows_pollution = cursor.fetchall()
        for Cidade, Região, Entidade, Qualidade_do_Ar, Poluição_da_Água in rows_pollution:
            print(f'A Cidade é: {Cidade} ')
            print(f'A Região é: {Região} ')
            print(f'A Entidade é: {Entidade} ')
            print(f'A Qualidade_do_Ar é: {Qualidade_do_Ar} ')
            print(f'A Poluição da Água é: {Poluição_da_Água}\n')

        # Lendo dados da tabela T_PRODUCTION
        cursor.execute("SELECT * FROM T_PRODUCTION")
        rows_production = cursor.fetchall()
        for Entidade, Ano, Produção_Anual_Plástico in rows_production:
            print(f'A Entidade é: {Entidade} ')
            print(f'O Ano é: {Ano} ')
            print(f'A Produção Anual de Plástico é: {Produção_Anual_Plástico}\n')

    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"An error occurred: {error.code} - {error.message}")
    finally:
        cursor.close()
        resposta_user = input("Deseja fazer mais alguma ação? (SIM/NÃO)").upper()
        if resposta_user == 'SIM':
            return inicio()
        else:
            return final()
def update():
    connection = oracledb.connect(user=USER, password=PASS, dsn=dsnStr)
    cursor = connection.cursor()
        
    escolha_table = input("De qual o nome da tabela que você deseja alterar? Para Produção de Plástico digite (PP) e para Poluição da água digite (PA)").lower()
    try:        
        if escolha_table == 'pp':
            user = input("O qual opção deseja mudar? (Entidade,Ano,Produção Anual de Plástico): ")
            ano = input("Selecione qual ano deseja alterar: ")
            new_value = input(f"Digite o novo valor para da(o){user} do ano de {ano}: ")
        
        
            if user.lower() == 'entidade':
                sql = "UPDATE T_PRODUCTION SET entidade = :1 WHERE ano = :2"
            elif user.lower() == 'ano':
                sql = "UPDATE T_PRODUCTION SET ano = :1 WHERE ano = :2"
            elif user.lower() == 'produção anual de plástico':
                sql = "UPDATE T_PRODUCTION SET produção_anual_de_plástico = :1 WHERE ano = :2"
            else:
                print("Opção inválida. Por favor, escolha um dos campos listados.")
                return update()
            cursor.execute(sql, [new_value, ano])
            
        elif escolha_table == 'pa':
            user = input("O qual opção deseja mudar? (Cidade, Região, Entidade, Qualidade do Ar, Poluição da Água): ")
            cidade = input("Selecione qual cidade deseja alterar: ")
            new_value = input(f"Digite o novo valor para da(o){user} do ano de {cidade}: ")
            
            if user.lower() == 'cidade':
                sql = "UPDATE T_POLLUTION SET cidade = :1 WHERE cidade = :2"
            elif user.lower() == 'região' or "regiao":
                sql = "UPDATE T_POLLUTION SET regiao = :1 WHERE cidade = :2"
            elif user.lower() == 'entidade':
                sql = "UPDATE T_POLLUTION SET entidade = :1 WHERE cidade = :2"
            elif user.lower() == 'qualidade do ar':
                sql = "UPDATE T_POLLUTION SET Qualidade_do_Ar = :1 WHERE cidade = :2"
            elif user.lower() == 'poluição da água':
                sql = "UPDATE T_POLLUTION SET Poluição_da_Água = :1 WHERE cidade = :2"
            else:
                print("Opção inválida. Por favor, escolha um dos campos listados.")
                return update()
            cursor.execute(sql, [new_value, ano])
            
    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"An error occurred: {error.code} - {error.message}")
    finally:
        cursor.close()
        resposta_user = input("Deseja fazer mais alguma ação? (SIM/NÃO)").upper()
        if resposta_user == 'SIM':
            return inicio()
        else:
            return final()
def final():
    print("Muito Obrigado pela preferência até logo!")
    SystemExit()
           
if __name__ == "__main__":
    with open("credenciais.txt", 'r') as f:
        credenciais = {}
        for line in f:
            key, value = line.strip().split("=")
            credenciais[key] = value
    
    USER = credenciais["user"]
    PASS = credenciais["pass"]

    dsnStr = oracledb.makedsn("oracle.fiap.com.br", 1521, "ORCL")
    iniciar()

____________________________________________________________________

Bem Vindo a ______ o que deseja fazer?
1 - Deseja adicionar a planilha inteira?
2 - Deseja adicionar uma informação?
3 - Deseja atualizar alguma tabela?
4 - Deseja visualizar alguma tabela?
5 - Deseja excluir alguma tabela?
6 - Sair
____________________________________________________________________

O número escolhido foi 1. Executando...




NameError: name 'inicio' is not defined